In [1]:
# Import source

using Base.Test
using Plots
gr()
using Distributions

cd("../Source")
include("ChiralMC.jl")
using ChiralMC

include("functions_msc.jl")
include("functions_observables.jl")


cd("../Notebooks/")

In [18]:
include("../Source/functions_energy.jl")

ΔE (generic function with 1 method)

In [2]:
function testEnDiff(ψ₀::State, ψ::State, dE::Float64)
    E₀ = E(ψ₀)
    E′ = E(ψ)
    if E₀ > E′
        same = isapprox(E′-dE, E₀; atol=0, rtol=1e-13)
    else
        same = isapprox(E′, E₀+dE; atol=0, rtol=1e-13)
    end
    if !same
        println("|(E′-E₀)-dE|/max(E′-E₀, dE) = $(abs((E′-E₀)-dE)/max(E′-E₀, dE))")
        println("E₀\t\t\tΔE\t\t\tE′-E₀\t\t\t|ΔE-E₁-E₂|")
        println("$E₀\t$dE\t$(E′-E₀)\t$(abs(dE - (E′-E₀)))")
    end
    return same
end

testEnDiff (generic function with 1 method)

In [6]:
g = 0.3; ν = 0.3; L = 12; T = 0.12; γ = 1.0; κ₅ = 1.0; f = 1.4/L#0.12/L
β = 1/T
# Create system
syst = SystConstants(L, L, γ, 1/g^2, ν, κ₅, f, β)
ψ₀ = State(1, syst)
ψ = copy(ψ₀)
dE = 0.0
for i = 1:20
    dE += mcSweepEn!(ψ)
end
E′ = E(ψ)
E⁰ = E(ψ₀)
println("dE =\t$(dE)\nE-E₀ =\t$(E(ψ)-E(ψ₀))\nE₀ =\t$(E(ψ₀))\n|dE - (E-E₀)| =\t$(abs(dE-(E′-E⁰)))")

dE =	-1018.8506457968613
E-E₀ =	-1018.8506457968747
E₀ =	-3936.995989561487
|dE - (E-E₀)| =	1.3415046851150692e-11


In [4]:
println("\nTesting Energy difference function ΔE\n----------------------------------------------------------------")
# Test case. We make a random 3x3 lattice, and put two different lattice sites in the middle
# i.e. the [2,2] position. Then we use the different functions to calculate the energy
# difference associated with this change.

L = 5
ψ₂ = State(1,L)
ϕ′ = LatticeSite() # Get random lattice site
ψ₁ = copy(ψ₂)
@test ψ₁.nb[3,2,2].ϕᵣ₊₂.u⁺ == ψ₁.lattice[2,2,2].u⁺
@test ψ₁.nb[3,2,2].ϕᵣ₊₂ == ψ₁.lattice[2,2,2]
set!(ψ₁.lattice[2,2,2], ϕ′)
@test ψ₁.nb[3,2,2].ϕᵣ₊₂.u⁺ == ϕ′.u⁺

dE = ΔE(ϕ′,ψ₂.lattice[2,2,2],ψ₂.nb[2,2,2],ψ₂.nnb[2,2,2],ψ₂.nnnb[2,2,2], 2,ψ₂.consts)
println("Checking that ΔE and E get same result")
println(@test testEnDiff(ψ₂, ψ₁, dE))
#println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13))



Testing Energy difference function ΔE
----------------------------------------------------------------
Checking that ΔE and E get same result
Test Passed


In [5]:
g = 0.3; ν = 0.3; L = 12; T = 0.12; γ = 1.0; κ₅ = 1.0; f = 1.4/L#0.12/L
β = 1/T
# Create system
syst = SystConstants(L, L, γ, 1/g^2, ν, κ₅, f, β)
ψ₀ = State(2, syst)
println("Checking that ΔE gives same result as E'-E on all different sites of the lattice")
ψ = copy(ψ₀)
for z_pos = 1:L, h_pos = 1:L, v_pos = 1:L
    ψ = copy(ψ₀)
    ϕ′ = LatticeSite()
    dE = ΔE(ϕ′, ψ₀.lattice[v_pos,h_pos,z_pos],ψ₀.nb[v_pos,h_pos,z_pos],ψ₀.nnb[v_pos,h_pos,z_pos],ψ₀.nnnb[v_pos,h_pos,z_pos],h_pos,ψ₀.consts)
    set!(ψ.lattice[v_pos,h_pos,z_pos], ϕ′)
    if !testEnDiff(ψ₀, ψ, dE)
        println("[$(v_pos), $(h_pos), $(z_pos)]")
    end
    #@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13)
end
println(@test isapprox(E(ψ)-dE, E(ψ₀); atol=0, rtol=1e-13))



Checking that ΔE gives same result as E'-E on all different sites of the lattice
Test Passed


In [6]:
println("\nTesting Energy difference function ΔE\n----------------------------------------------------------------")
# Test case. We make a random 5x5 lattice, and put two different lattice sites in the middle
# i.e. the [2,2] position. Then we use the different functions to calculate the energy
# difference associated with this change.

L = 5
ψ₂ = State(2,L)
site = LatticeSite() # Get random lattice site
ψ₁ = copy(ψ₂)
@test ψ₁.nb[3,2].ϕᵣ₊₂.u⁺ == ψ₁.lattice[2,2].u⁺
@test ψ₁.nb[3,2].ϕᵣ₊₂ == ψ₁.lattice[2,2]
set!(ψ₁.lattice[2,2], site)
@test ψ₁.nb[3,2].ϕᵣ₊₂.u⁺ == site.u⁺

dE = ΔE(site,ψ₂.lattice[2,2],ψ₂.nb[2,2],ψ₂.nnb[2,2],ψ₂.nnnb[2,2], 2,ψ₂.consts)
println("Checking that ΔE and E get the same result")
println("ΔE\t\t\tE₁-E₂")
println("$dE\t$(E(ψ₁)-E(ψ₂))")
println(dE/(E(ψ₁)-E(ψ₂)))
println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13))


Testing Energy difference function ΔE
----------------------------------------------------------------
Checking that ΔE and E get the same result
ΔE			E₁-E₂
470.02615900889134	470.0261590088908
1.000000000000001
Test Passed


In [7]:

println("Checking that ΔE gives same result as E'-E on all different sites of the lattice")
println("[v_pos, h_pos]\t|ΔE-(E₁-E₂)|")
for h_pos = 1:L, v_pos = 1:L
    ψ₁ = copy(ψ₂)
    ϕ′ = LatticeSite()
    dE = ΔE(ϕ′, ψ₂.lattice[v_pos,h_pos],ψ₂.nb[v_pos,h_pos],ψ₂.nnb[v_pos,h_pos],ψ₂.nnnb[v_pos,h_pos],h_pos,ψ₂.consts)
    set!(ψ₁.lattice[v_pos,h_pos], ϕ′)
    println("[$(v_pos), $h_pos]\t\t$(abs(dE-(E(ψ₁)-E(ψ₂))))")
    @test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=5e-13)
end
println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=5e-13))


Checking that ΔE gives same result as E'-E on all different sites of the lattice
[v_pos, h_pos]	|ΔE-(E₁-E₂)|
[1, 1]		0.0
[2, 1]		3.979039320256561e-13
[3, 1]		4.547473508864641e-13
[4, 1]		1.7053025658242404e-13
[5, 1]		8.526512829121202e-14
[1, 2]		5.684341886080802e-14
[2, 2]		4.547473508864641e-13
[3, 2]		2.842170943040401e-14
[4, 2]		1.4210854715202004e-13
[5, 2]		4.547473508864641e-13
[1, 3]		4.547473508864641e-13
[2, 3]		3.410605131648481e-13
[3, 3]		5.684341886080801e-13
[4, 3]		5.684341886080801e-13
[5, 3]		2.2737367544323206e-13
[1, 4]		5.684341886080802e-14
[2, 4]		5.684341886080801e-13
[3, 4]		2.1316282072803006e-13
[4, 4]		0.0
[5, 4]		1.9895196601282805e-13
[1, 5]		3.410605131648481e-13
[2, 5]		7.958078640513122e-13
[3, 5]		2.842170943040401e-14
[4, 5]		1.8474111129762605e-13
[5, 5]		4.547473508864641e-13
Test Passed
